In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()


True

In [2]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

## Langsmith Tracking and Tracing
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2', 'true')

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="o1-mini")
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x1097c03e0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x109cb86e0> root_client=<openai.OpenAI object at 0x109646de0> root_async_client=<openai.AsyncOpenAI object at 0x1097c04a0> model_name='o1-mini' temperature=1.0 model_kwargs={} openai_api_key=SecretStr('**********')


In [4]:
result = llm.invoke("What is the capital of France?")

In [5]:
print(result.content)

The capital of France is Paris.


In [6]:
from langchain_groq import ChatGroq
model=ChatGroq(model="qwen-qwq-32B")
model.invoke("What is the capital of France?")

AIMessage(content="\n<think>\nOkay, the user is asking for the capital of France. Let me think. I know that France is a country in Western Europe. I remember from school that the capital cities are usually where the government is based. Paris comes to mind. Wait, isn't Paris the capital? Let me confirm. Yeah, I've heard about the Eiffel Tower and the Louvre, which are in Paris. But I should make sure there hasn't been any recent changes. No, capitals don't change that often. Also, checking in my mind, other major cities in France like Marseille or Lyon, but they're not capitals. So yes, the answer should be Paris. I don't think there's any trick here. The user probably wants a straightforward answer. Just answer Paris.\n</think>\n\nThe capital of France is **Paris**. Located in the north-central part of the country, it is not only the political and cultural heart of France but also a global center for art, fashion, and cuisine. Iconic landmarks such as the Eiffel Tower, Louvre Museum, 

In [7]:
### Prompt Engineering

from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert AI Engineer. Provide me an answer to the question."),
    ("user", "{input}"),
])

In [8]:
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI Engineer. Provide me an answer to the question.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [9]:
from langchain_groq import ChatGroq
model=ChatGroq(model="gemma2-9b-it")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10a2c7140>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10a2c6900>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [10]:
### chaining
chain=prompt|model
chain


ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI Engineer. Provide me an answer to the question.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10a2c7140>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10a2c6900>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [11]:
response=chain.invoke({"input": "What is the capital of France?"})
print(response.content)

The capital of France is **Paris**. 😊  



In [12]:
### Output Parser

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | model | output_parser

response=chain.invoke({"input": "Can you tell me about Langsmith?"})
print(response)

You're asking about **Langsmith**, which is an exciting project in the world of AI! 

Langsmith is an open-source platform developed by the **Hugging Face** team. Think of it as a powerful toolkit specifically designed for fine-tuning and deploying large language models (LLMs). 

Here's a breakdown of what makes Langsmith special:

**Key Features:**

* **Simplified Fine-Tuning:** Langsmith streamlines the process of customizing existing LLMs for specific tasks. It provides a user-friendly interface and intuitive workflows, making fine-tuning accessible even to those without deep technical expertise.
* **Efficient Training:** It leverages techniques like gradient accumulation and mixed precision training to optimize the fine-tuning process, reducing training time and resource consumption.
* **Collaborative Development:** As an open-source project, Langsmith encourages community involvement and contributions.  Developers can share their fine-tuned models, datasets, and training scripts, 

In [13]:
from langchain_core.output_parsers import JsonOutputParser

output_parser = JsonOutputParser()
output_parser.get_format_instructions()

'Return a JSON object.'

In [14]:
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate

output_parser = JsonOutputParser()

prompt = PromptTemplate(
    template="Answer the user query \n {format_instruction}\n {query}\n",
    input_variables=["query"],
    partial_variables={"format_instruction": output_parser.get_format_instructions},
)

In [15]:
prompt


PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instruction': <bound method JsonOutputParser.get_format_instructions of JsonOutputParser()>}, template='Answer the user query \n {format_instruction}\n {query}\n')

In [16]:
chain=prompt | model | output_parser
response=chain.invoke({"query": "Can you tell me about Langsmith?"})
print(response)

{'name': 'Langsmith', 'description': 'Langsmith is a powerful open-source platform designed for developing and deploying large language models (LLMs). It provides a comprehensive suite of tools and resources to streamline the entire LLM lifecycle, from training and fine-tuning to deployment and monitoring.', 'key_features': ['Modular and extensible architecture', 'Support for various LLM frameworks (e.g., Transformers, 🤗)', 'Efficient training and inference pipelines', 'Built-in tools for model evaluation and debugging', 'Simplified deployment options for cloud and on-premise environments', 'Active community and open-source contributions'], 'website': 'https://github.com/langs-org/langs'}


In [24]:
##output parser
#from langchain_core.output_parsers import XMLOutputParser
from langchain.output_parsers.xml import XMLOutputParser

# XML Output Parser
output_parser = XMLOutputParser()

# Prompt that instructs the model to return XML
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Respond in this XML format: <response><answer>Your answer here</answer></response>"),
    ("human", "{input}")
])

# Build the chain
chain = prompt | model

# Run the chain
#response = chain.invoke({"input": "What is LangChain?"})

raw_output =chain.invoke({"input": "What is LangChain?"})

# Print result
print(raw_output)



content='<response><answer>LangChain is an open-source framework for developing applications powered by language models. It provides tools and components for tasks like text generation, question answering, summarization, and more.</answer></response>\n' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 39, 'total_tokens': 87, 'completion_time': 0.087272727, 'prompt_time': 0.002365536, 'queue_time': 0.090673362, 'total_time': 0.089638263}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run--70620c68-eb3a-493d-9728-5c1db8a07e3d-0' usage_metadata={'input_tokens': 39, 'output_tokens': 48, 'total_tokens': 87}


In [25]:
## With Pydantic
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

model = ChatOpenAI(temperature=0.7)


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

{'setup': "Why couldn't the bicycle stand up by itself?",
 'punchline': 'Because it was two tired.'}

In [28]:
### Assignment --- Chat prompt template


from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# 1. Instantiate your LLM
model = ChatOpenAI(temperature=0.0)  # deterministic output for structured tasks

# 2. Define the Pydantic model for your product schema
class Product(BaseModel):
    name: str = Field(..., description="The product's official name")
    details: str = Field(..., description="A brief description of the product")
    price_usd: float = Field(..., description="The price of the product in US dollars")

# 3. Create a JsonOutputParser around that model
parser = JsonOutputParser(pydantic_object=Product)

# 4. Build a PromptTemplate that injects the parser's format instructions
prompt = PromptTemplate(
    template=(
        "You are a helpful assistant that, given a product, returns valid JSON "
        "conforming to the following schema:\n\n"
        "{format_instructions}\n\n"
        "Product: {product}\n"
    ),
    input_variables=["product"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 5. Compose your chain: prompt → model → parser
chain = prompt | model | parser

# 6. Invoke with a product query
result: Product = chain.invoke({"product": "iPhone 15 Pro Max"})
print(result)



{'name': 'iPhone 15 Pro Max', 'details': 'The latest flagship smartphone from Apple with advanced features and technology.', 'price_usd': 1299.99}
